In [ ]:
conda install -c conda-forge ipywidgets

In [2]:
import pandas as pd
import folium as fo
import re
import bs4 
import requests 
import ipywidgets as widgets

In [3]:
def get_data():  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    pagina = requests.get('http://queimadas.dgi.inpe.br/queimadas/dados-abertos/')
    content = pagina.content
    soup = bs4.BeautifulSoup(content)

    lista_regiao_url = []
    lista_regiao = []
    
    for tr in soup.findAll('tr'):                        
        regiao_url = []        
        regiao_nome = []        
        for td in tr.findAll('td'):                        
            
            regiao_nome.append(td.get_text().strip())                                    
            for a in td.find_all('a'):                
                if (len(regiao_nome)==4):   #dados csv 48h                  
                    regiao_url.append(a.get('href'))                    
            
        lista_regiao_url.append([regiao_nome[0],regiao_url[0]])
        lista_regiao.append(regiao_nome[0])
        
    return lista_regiao_url,  lista_regiao
    
lista = get_data()

In [13]:
print('Selecione a regiao que deseja inspecionar os focos de incendio:')
ToggleButtons_regiao = widgets.ToggleButtons(
    options=lista[1],    
    value=lista[1][1],    
    disabled=False    
)

display(ToggleButtons_regiao)

Selecione a regiao que deseja inspecionar os focos de incendio:


ToggleButtons(index=1, options=('América do Sul', 'Brasil', 'Acre', 'Alagoas', 'Amapá', 'Amazonas', 'Bahia', '…

In [14]:
idx = lista[1].index(ToggleButtons_regiao.value)

In [15]:
print(idx)

14


In [16]:
url = 'http://queimadas.dgi.inpe.br' +  str(lista[0][idx][1])
print(url)

http://queimadas.dgi.inpe.br/queimadas/dados-abertos/download/?utm_campaign=dados-abertos&outputFormat=csv&utm_medium=landing-page&time=48h&utm_content=focos_brasil_mg_48h&id=focos_brasil_mg&utm_source=landing-page


In [17]:
CSV_URL = url
#'http://queimadas.dgi.inpe.br/queimadas/dados-abertos/download/?utm_campaign=dados-abertos&outputFormat=csv&utm_medium=landing-page&time=48h&utm_content=focos_brasil_48h&id=focos_brasil&utm_source=landing-page'

In [18]:
df = pd.read_csv(CSV_URL)

In [19]:
df.head()

,FID,id,longitude,latitude,data_hora_gmt,satelite,municipio,estado,pais,municipio_id,estado_id,pais_id,numero_dias_sem_chuva,precipitacao,risco_fogo,bioma,geom
0,focos_brasil_ontem_hoje_ref.2ea44887-884d-3140...,2ea44887-884d-3140-85a8-b70658148375,-42.887,-20.414,2020-09-20T16:55:00,AQUA_M-T,PONTE NOVA,MINAS GERAIS,Brasil,3152105,31,33,NaN,NaN,NaN,Mata Atlântica,POINT (-20.414 -42.887)
1,focos_brasil_ontem_hoje_ref.b6711853-fc3b-3485...,b6711853-fc3b-3485-9b76-078da690a751,-41.804,-15.151,2020-09-20T16:55:00,AQUA_M-T,SÃO JOÃO DO PARAÍSO,MINAS GERAIS,Brasil,3162708,31,33,NaN,NaN,NaN,Mata Atlântica,POINT (-15.151 -41.804)
2,focos_brasil_ontem_hoje_ref.e9165555-fe75-386d...,e9165555-fe75-386d-bac7-981a8fcddee2,-41.811,-15.150,2020-09-20T16:55:00,AQUA_M-T,SÃO JOÃO DO PARAÍSO,MINAS GERAIS,Brasil,3162708,31,33,NaN,NaN,NaN,Mata Atlântica,POINT (-15.15 -41.811)
3,focos_brasil_ontem_hoje_ref.7836c4b2-4519-311c...,7836c4b2-4519-311c-b5ea-bb16f10f2bbe,-41.785,-15.148,2020-09-20T16:55:00,AQUA_M-T,NINHEIRA,MINAS GERAIS,Brasil,3144656,31,33,NaN,NaN,NaN,Mata Atlântica,POINT (-15.148 -41.785)
4,focos_brasil_ontem_hoje_ref.000f7c09-0e1f-3dfb...,000f7c09-0e1f-3dfb-b01d-cd47fa3cf755,-41.792,-15.147,2020-09-20T16:55:00,AQUA_M-T,SÃO JOÃO DO PARAÍSO,MINAS GERAIS,Brasil,3162708,31,33,NaN,NaN,NaN,Mata Atlântica,POINT (-15.147 -41.792)


In [20]:
def replace(match):
    data = {"á": "a", "ã" : "a", "Á": "A","Ã" : "A", "ç": "c", "ê": "e", "Ô":"O"}
    return data.get(match.group(0))

In [21]:
titulo = 'Queimadas - '+ lista[0][idx][0] 

map = fo.Map(location=[-23.5489, -46.6388], 
             title= titulo,
             zoom_start=5,
             width=900, 
             height=1200)
po = fo.FeatureGroup(name='Queimadas')

for row in range(len(df)):
    po.add_child(fo.Marker(location=[ df['latitude'][row], df['longitude'][row]],
                           popup=[re.sub(u'[^a-zA-Z0-9: ]', replace, df['municipio'][row]),
                                  re.sub(u'[^a-zA-Z0-9: ]', replace, df['estado'][row])],
                           icon=fo.Icon(color='red')))    
map.add_child(po)    